# Imports


In [17]:
import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import evaluate

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score

from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)


# Load dataset

In [18]:
data = pd.read_csv('ScamDataset.csv', delimiter=',', names=['message', 'label'])

original_shape = data.shape

# Removing duplicate rows
data = data.drop_duplicates()

# Shape after removing duplicates
new_shape = data.shape

original_shape, new_shape

((20670, 2), (15241, 2))

In [19]:
data = data.sample(frac=1).reset_index(drop=True)


# Preprocess

In [20]:
#data['label'] = data['label'].map({'normal': 0, 'fraud': 1})
data['label'] = data['label'].astype('category')
data['target'] = data['label'].cat.codes
data


,message,label,target
0,Who u talking about?,normal,1
1,suspending your social security number because...,fraud,0
2,Hi:)cts employee how are you?,normal,1
3,Important notice: Your tax refund requires urg...,fraud,0
4,Important notice: Your prize claim requires ur...,fraud,0
...,...,...,...
15236,Great. So should i send you my account number.,normal,1
15237,Important notice: Your account verification re...,fraud,0
15238,This is Ms. Morgan. I was trying to reach you ...,fraud,0
15239,Important notice: Your prize claim requires ur...,fraud,0


In [21]:
# Decode
data['label'].cat.categories
category_map = {code: category for code, category in enumerate(data['label'].cat.categories)}
category_map

{0: 'fraud', 1: 'normal'}

In [22]:
train_end_point = int(data.shape[0]*0.6)
val_end_point = int(data.shape[0]*0.8)
df_train = data.iloc[:train_end_point,:]
df_val = data.iloc[train_end_point:val_end_point,:]
df_test = data.iloc[val_end_point:,:]
print(df_train.shape, df_test.shape, df_val.shape)

(9144, 3) (3049, 3) (3048, 3)


### Convert from Pandas DataFrame to Hugging Face Dataset
* Also let's shuffle the training set.
* We put the components train,val,test into a DatasetDict so we can access them later with HF trainer.
* Later we will add a tokenized dataset


In [23]:
# Converting pandas DataFrames into Hugging Face Dataset objects:
dataset_train = Dataset.from_pandas(df_train.drop('label',axis=1))
dataset_val = Dataset.from_pandas(df_val.drop('label',axis=1))
dataset_test = Dataset.from_pandas(df_test.drop('label',axis=1))


In [24]:
# Shuffle the training dataset
dataset_train_shuffled = dataset_train.shuffle(seed=42)  # Using a seed for reproducibility

In [25]:
# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': dataset_train_shuffled,
    'val': dataset_val,
    'test': dataset_test
})
dataset

DatasetDict({
    train: Dataset({
        features: ['message', 'target'],
        num_rows: 9144
    })
    val: Dataset({
        features: ['message', 'target'],
        num_rows: 3048
    })
    test: Dataset({
        features: ['message', 'target'],
        num_rows: 3049
    })
})

In [26]:
dataset['train']

Dataset({
    features: ['message', 'target'],
    num_rows: 9144
})

In [27]:
df_train.target.value_counts(normalize=True)

target
0    0.535871
1    0.464129
Name: proportion, dtype: float64

In [28]:
class_weights=(1/df_train.target.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights


tensor([0.4641, 0.5359])

In [29]:
model_name = "meta-llama/Meta-Llama-3-8B"

In [30]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

In [31]:
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=2
)

model

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.64s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [33]:
model = prepare_model_for_kbit_training(model)
model

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [34]:
model = get_peft_model(model, lora_config)
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
        

#Load Tokenizer

In [35]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [36]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

# Loop through dataset to measure performance before training/fitting the model

In [37]:
messages = df_test.message.tolist()
messages

['22 days to kick off! For Euro2004 you will be kept up to date with the latest news and results daily. To be removed send GET TXT STOP to 83222',
 'you are sweet as well, princess. Please tell me your likes and dislikes in bed...',
 'Important notice: Your prize claim requires urgent attention. Act immediately to avoid issues. Ref: 9436',
 'But mother we haven’t experienced it.',
 'Dear 7977XXXX34,  WIN 1st Prize of Rs. 2 Lakhs! Create your team for IND vs ENG 1st ODI on Paytm First Games & Win Up to Rs. 10 Cr. Click m.paytm.me/fnpt T&C.',
 'Important notice: Your account verification requires urgent attention. Act without delay to avoid issues. Ref: 5496',
 'Important notice: Your account verification requires urgent attention. Act immediately to avoid issues. Ref: 3958',
 "I'm calling from the National Survey Department. We are conducting a survey about public transportation usage and would like to know your views. The survey takes about 10 minutes, and your responses will help impr

In [38]:
# Convert summaries to a list
messages = df_test.message.tolist()

# Define the batch size
batch_size = 32  # You can adjust this based on your system's memory capacity

# Initialize an empty list to store the model outputs
all_outputs = []

# Process the sentences in batches
for i in range(0, len(messages), batch_size):
    # Get the batch of sentences
    batch_messages = messages[i:i + batch_size]

    # Tokenize the batch
    inputs = tokenizer(batch_messages, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move tensors to the device where the model is (e.g., GPU or CPU)
    inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

    # Perform inference and store the logits
    with torch.no_grad():
        outputs = model(**inputs)
        all_outputs.append(outputs['logits'])

    print(batch_messages)


c:\Users\Potbottom\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


['22 days to kick off! For Euro2004 you will be kept up to date with the latest news and results daily. To be removed send GET TXT STOP to 83222', 'you are sweet as well, princess. Please tell me your likes and dislikes in bed...', 'Important notice: Your prize claim requires urgent attention. Act immediately to avoid issues. Ref: 9436', 'But mother we haven’t experienced it.', 'Dear 7977XXXX34,  WIN 1st Prize of Rs. 2 Lakhs! Create your team for IND vs ENG 1st ODI on Paytm First Games & Win Up to Rs. 10 Cr. Click m.paytm.me/fnpt T&C.', 'Important notice: Your account verification requires urgent attention. Act without delay to avoid issues. Ref: 5496', 'Important notice: Your account verification requires urgent attention. Act immediately to avoid issues. Ref: 3958', "I'm calling from the National Survey Department. We are conducting a survey about public transportation usage and would like to know your views. The survey takes about 10 minutes, and your responses will help improve bus

In [39]:
final_outputs = torch.cat(all_outputs, dim=0)
final_outputs

tensor([[-1.8261,  0.7307],
        [-5.2043,  2.9651],
        [-1.7140,  4.8460],
        ...,
        [-3.7808, -1.1111],
        [-2.3647,  5.2283],
        [ 1.0745,  2.7373]], device='cuda:0')

In [40]:
final_outputs.argmax(axis=1)

tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0')

In [41]:
df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()
df_test['predictions']

C:\Users\Potbottom\AppData\Local\Temp\ipykernel_17744\1915017267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()


12192    1
12193    1
12194    1
12195    1
12196    1
        ..
15236    1
15237    1
15238    1
15239    1
15240    1
Name: predictions, Length: 3049, dtype: int64

In [42]:
df_test['predictions'].value_counts()

predictions
1    2905
0     144
Name: count, dtype: int64

In [43]:
df_test['predictions']=df_test['predictions'].apply(lambda l:category_map[l])
df_test['predictions']

C:\Users\Potbottom\AppData\Local\Temp\ipykernel_17744\4293232383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions']=df_test['predictions'].apply(lambda l:category_map[l])


12192    normal
12193    normal
12194    normal
12195    normal
12196    normal
          ...  
15236    normal
15237    normal
15238    normal
15239    normal
15240    normal
Name: predictions, Length: 3049, dtype: object

# Evaluate Performance

In [44]:
def get_performance_metrics(df_test):
  y_test = df_test.label
  y_pred = df_test.predictions

  print("Confusion Matrix:")
  print(confusion_matrix(y_test, y_pred))

  print("\nClassification Report:")
  print(classification_report(y_test, y_pred))

  print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
  print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Results without fined tune


In [45]:
get_performance_metrics(df_test)

Confusion Matrix:
[[  46 1717]
 [  98 1188]]

Classification Report:
              precision    recall  f1-score   support

       fraud       0.32      0.03      0.05      1763
      normal       0.41      0.92      0.57      1286

    accuracy                           0.40      3049
   macro avg       0.36      0.47      0.31      3049
weighted avg       0.36      0.40      0.27      3049

Balanced Accuracy Score: 0.4749433005560118
Accuracy Score: 0.4047228599540833


In [46]:
MAX_LEN = 512
col_to_delete = ['message']

def llama_preprocessing_function(examples):
    return tokenizer(examples['message'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_datasets = tokenized_datasets.rename_column("target", "label")
tokenized_datasets.set_format("torch")

Map: 100%|██████████| 3049/3049 [00:00<00:00, 62837.79 examples/s]


In [47]:
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)


In [48]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),'accuracy':accuracy_score(predictions,labels)}

# Define Custom trainer


In [49]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels and convert them to long type for cross_entropy
        labels = inputs.pop("labels").long()

        # Forward pass
        outputs = model(**inputs)

        # Extract logits assuming they are directly outputted by the model
        logits = outputs.get('logits')

        # Compute custom loss with class weights for imbalanced data handling
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [50]:
training_args = TrainingArguments(
    output_dir = 'sentiment_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 7,
    weight_decay = 0.01,
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

In [51]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['val'],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    class_weights=class_weights,
)

C:\Users\Potbottom\AppData\Local\Temp\ipykernel_17744\2851958097.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)


In [52]:
train_result = trainer.train()

  0%|          | 0/8001 [00:00<?, ?it/s]c:\Users\Potbottom\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
  6%|▌         | 500/8001 [14:15<3:13:32,  1.55s/it]

{'loss': 0.0789, 'grad_norm': 9.785673000806128e-07, 'learning_rate': 9.375078115235596e-05, 'epoch': 0.44}


 12%|█▏        | 1000/8001 [36:19<2:42:02,  1.39s/it]  

{'loss': 0.0436, 'grad_norm': 3.3324738524243e-08, 'learning_rate': 8.750156230471191e-05, 'epoch': 0.87}


                                                     
 14%|█▍        | 1143/8001 [44:08<2:53:40,  1.52s/it]

...

{'loss': 0.0012, 'grad_norm': 3.818229743046686e-06, 'learning_rate': 1.2498437695288089e-08, 'epoch': 7.0}


                                                     
100%|██████████| 8001/8001 [5:08:59<00:00,  1.67s/it]

{'eval_loss': 0.07363533228635788, 'eval_balanced_accuracy': 0.9931141899720405, 'eval_accuracy': 0.9934383202099738, 'eval_runtime': 217.4702, 'eval_samples_per_second': 14.016, 'eval_steps_per_second': 1.752, 'epoch': 7.0}


c:\Users\Potbottom\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 8001/8001 [5:09:01<00:00,  2.32s/it]

{'train_runtime': 18541.2401, 'train_samples_per_second': 3.452, 'train_steps_per_second': 0.432, 'train_loss': 0.01167884346692894, 'epoch': 7.0}


In [54]:
def make_predictions(model,df_test):


  # Convert summaries to a list
  messages = df_test.message.tolist()

  # Define the batch size
  batch_size = 32  # You can adjust this based on your system's memory capacity

  # Initialize an empty list to store the model outputs
  all_outputs = []

  # Process the sentences in batches
  for i in range(0, len(messages), batch_size):
      # Get the batch of sentences
      batch_messages = messages[i:i + batch_size]

      # Tokenize the batch
      inputs = tokenizer(batch_messages, return_tensors="pt", padding=True, truncation=True, max_length=512)

      # Move tensors to the device where the model is (e.g., GPU or CPU)
      inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

      # Perform inference and store the logits
      with torch.no_grad():
          outputs = model(**inputs)
          all_outputs.append(outputs['logits'])
      
      print(batch_messages)

  final_outputs = torch.cat(all_outputs, dim=0)
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()
  df_test['predictions']=df_test['predictions'].apply(lambda l:category_map[l])


make_predictions(model,df_test)

['22 days to kick off! For Euro2004 you will be kept up to date with the latest news and results daily. To be removed send GET TXT STOP to 83222', 'you are sweet as well, princess. Please tell me your likes and dislikes in bed...', 'Important notice: Your prize claim requires urgent attention. Act immediately to avoid issues. Ref: 9436', 'But mother we haven’t experienced it.', 'Dear 7977XXXX34,  WIN 1st Prize of Rs. 2 Lakhs! Create your team for IND vs ENG 1st ODI on Paytm First Games & Win Up to Rs. 10 Cr. Click m.paytm.me/fnpt T&C.', 'Important notice: Your account verification requires urgent attention. Act without delay to avoid issues. Ref: 5496', 'Important notice: Your account verification requires urgent attention. Act immediately to avoid issues. Ref: 3958', "I'm calling from the National Survey Department. We are conducting a survey about public transportation usage and would like to know your views. The survey takes about 10 minutes, and your responses will help improve bus

C:\Users\Potbottom\AppData\Local\Temp\ipykernel_17744\3895869390.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()
C:\Users\Potbottom\AppData\Local\Temp\ipykernel_17744\3895869390.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions']=df_test['predictions'].apply(lambda l:category_map[l])


In [58]:
get_performance_metrics(df_test)

Confusion Matrix:
[[1753   10]
 [  16 1270]]

Classification Report:
              precision    recall  f1-score   support

       fraud       0.99      0.99      0.99      1763
      normal       0.99      0.99      0.99      1286

    accuracy                           0.99      3049
   macro avg       0.99      0.99      0.99      3049
weighted avg       0.99      0.99      0.99      3049

Balanced Accuracy Score: 0.9909430853142486
Accuracy Score: 0.991472613971794


In [56]:
metrics = train_result.metrics
max_train_samples = len(dataset_train)
metrics["train_samples"] = min(max_train_samples, len(dataset_train))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =         7.0
  total_flos               = 143195886GF
  train_loss               =      0.0117
  train_runtime            =  5:09:01.24
  train_samples            =        9144
  train_samples_per_second =       3.452
  train_steps_per_second   =       0.432


In [59]:
trainer.save_model("saved_model")

c:\Users\Potbottom\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
